In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read datasets

In [2]:
bruker_json = pd.read_json('../input/raw-data/bruker.json', orient = 'index').transpose()
columns = bruker_json['columns'][:5].values
data = bruker_json['data']

In [3]:
bruker = pd.DataFrame(columns = columns)
for user in range(len(data)):
    bruker.loc[len(bruker)] = data[user]
bruker

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,None,45,6,92103
1,1,M,50,16,55405-2546
2,2,M,18,20,44089
3,3,M,None,1,33304
4,4,M,35,6,48105
...,...,...,...,...,...
6035,6036,M,45,0,61821
6036,6037,F,None,None,None
6037,6038,None,25,16,33301
6038,6039,M,35,14,92075


In [4]:
film = pd.read_excel('../input/raw-data/film.xlsx', sheet_name = 'film', index_col = 0)
film

,FilmID,Tittel,Sjanger
0,0,Autumn in New York (2000),Drama|Romance
1,1,"Vie est belle, La (Life is Rosey) (1987)",Comedy|Drama
2,2,Defying Gravity (1997),Drama
3,3,Ruthless People (1986),Comedy
4,4,Portraits Chinois (1996),Drama
...,...,...,...
3878,3948,Cat People (1982),Horror
3879,3949,"Saltmen of Tibet, The (1997)",Documentary
3880,3950,Bride of Re-Animator (1990),Comedy|Horror
3881,3951,True Lies (1994),Action|Adventure|Comedy|Romance


In [5]:
rangering = pd.read_csv('../input/raw-data/rangering.dat', sep = '::', engine = 'python')
rangering.columns =['BrukerID','FilmID','Rangering','Tidstempel']
rangering

,BrukerID,FilmID,Rangering,Tidstempel
0,0,1561,7,959441640.0
1,0,1540,6,959441640.0
2,0,88,5,959441640.0
3,0,620,8,959441640.0
4,0,3771,9,959442113.0
...,...,...,...,...
900182,6040,1153,4,976584194.0
900183,6040,3714,4,976584260.0
900184,6040,2834,5,976584260.0
900185,6040,48,5,976584300.0


# Find errors and missing values in datasets and fix those errors  
From the ReadMe file we know that:  
1) Many user-profiles have missing values  
2) Ratings before 31.07.2000 use a 10-star scale, these reviews must be converted to 5-star scale.  
3) Genres must be converted to a designated column with binary values.  
4) Some MovieID's are not valid  


In [6]:
# For user-profiles we have a bunch of 'none' to fill out
# For 'Kjønn' i would like to keep the values binary.
print(bruker['Kjonn'].unique())
bruker['Kjonn'] = bruker['Kjonn'].replace([None,'nan'],np.nan)
#Find the probability distribution among genders
gender_prob_dist = bruker.Kjonn.value_counts(normalize = True)
#Create array with Male and Female values based on prob dist
fill = np.random.choice(['M','F'],
                     p=[gender_prob_dist[0], gender_prob_dist[1]], 
                     size = len(bruker[bruker.Kjonn.isna()]))
#Fill all nan values with prob dist array
bruker.loc[bruker.Kjonn.isna(),'Kjonn'] = fill

[None 'M' 'F' nan]


In [7]:
# For the alder-column we fill all non-valid values with the last observed value, with a limit of 1
print(bruker['Alder'].unique())
bruker['Alder'] = bruker['Alder'].replace([None,'nan'],np.nan)
bruker.Alder.fillna(method = 'ffill', limit = 1, inplace = True)
# The remaining non-valid values are filled with the median value
bruker.Alder.fillna(bruker.Alder.median(), inplace = True)

[45.0 50.0 18.0 None 35.0 25.0 56.0 1.0 nan]


In [8]:
# For the jobb-column we fill all non-valid values with 0, which is pre-defined as 'not specified' in the readme
print(bruker['Jobb'].unique())
bruker['Jobb'] = bruker['Jobb'].replace([None,'nan'],0)

# For the postcode-column we fill with 0
bruker['Postkode'] = bruker['Postkode'].replace([None,'nan'],0)
bruker.set_index('BrukerID',inplace = True)

[6.0 16.0 20.0 1.0 14.0 0.0 4.0 19.0 7.0 17.0 15.0 None 12.0 5.0 9.0 2.0
 3.0 10.0 99.0 11.0 8.0 13.0 18.0 nan]


In [9]:
bruker.index = bruker.index.map(int)
bruker

,Kjonn,Alder,Jobb,Postkode
BrukerID,,,,
0,F,45.0,6.0,92103
1,M,50.0,16.0,55405-2546
2,M,18.0,20.0,44089
3,M,18.0,1.0,33304
4,M,35.0,6.0,48105
...,...,...,...,...
6036,M,45.0,0.0,61821
6037,F,45.0,0.0,0
6038,M,25.0,16.0,33301


In [10]:
# Find all ratings with 10-star scale. That is, find all ratings prior to 01.08.2000

#First we look for missing data
pd.isna(rangering['Tidstempel']).value_counts()
#fill with 0
#Create deep copy to analyse ratings with missing timestamps later on
rangering_copy = rangering.copy()
rangering['Tidstempel'].fillna(0, inplace = True)


In [11]:
# Adding new columns Date and Hour translating the Timestamp values.
date = []
hour = []
for tstemp in rangering.Tidstempel.values:
    t = pd.to_datetime(tstemp,unit = 's')
    date.append(t.date())
    hour.append(t.hour)
rangering['Date'] = date
rangering['Hour'] = hour

In [12]:
# Adding a new feature 'DayOrNight' where 1 is day and 0 is night
# I suspect those who watch movies during nighttime have different rating-habits and movie preferences than those who watch movies during daytime
rangering['DayOrNight'] = rangering.Hour.map(lambda x: 1 if (x>6 & x<23) else 0)
rangering.DayOrNight.value_counts()

1    715659
0    184528
Name: DayOrNight, dtype: int64

In [13]:
#Now we can get all ratings prior to 01.08.2000 with valid timestamps, and change the rating to a 5-star system
rangering.loc[(rangering['Date'] >= datetime.date(1970,1,1)) & (rangering['Date'] < datetime.date(2000,8,1)),'Rangering'] = np.ceil(rangering['Rangering']/2).astype(int)

In [14]:
#now all entries are in the 5-star range
print(rangering['Rangering'].value_counts())
#But what about 10-star scale ratings with invalid timestamps with ratings less than 5?
#We need to find out if they exist

4    313561
3    235005
5    203445
2     97538
1     50638
Name: Rangering, dtype: int64


In [15]:
inv_timestamp = rangering_copy[pd.isna(rangering_copy['Tidstempel'])]
inv_timestamp['Rangering'].value_counts()
# For values 1-5 the distribution look very similar to the cleaned table.
# I want to assume that there are no 10-star scale ratings with invalid timestamps with ratings less than 5
# I normalize the distribution of the 1-5 star ratings with invalid timestamps, and compare with the normalized distribution of the cleaned table
# to see if the distribution is similiar
inv_5star = inv_timestamp[inv_timestamp['Rangering'] < 6]
print('Distribution of ratings in the 1-5 range with invalid timestamps \n',inv_5star['Rangering'].value_counts(normalize = True))
print('\n Distribution of ratings from cleaned table \n' ,rangering['Rangering'].value_counts(normalize = True))
# the normalized distributions confirms my assumption, so I assume that there are no 10-star scale ratings with invalid timestamps with ratings less than 5
# Therefore I only convert the ratings with invalid timestamps and a rating greater than 5


Distribution of ratings in the 1-5 range with invalid timestamps 
 4    0.343402
3    0.249603
5    0.247218
2    0.103339
1    0.056439
Name: Rangering, dtype: float64

 Distribution of ratings from cleaned table 
 4    0.348329
3    0.261062
5    0.226003
2    0.108353
1    0.056253
Name: Rangering, dtype: float64


In [16]:
#Removing the 'Date' column so that the structure match that of sample_data for further analysis
#I havent decided if i want to further explore the Date column, so I store a deep copy just in case.
rangering_modified = rangering.copy().drop(columns = ['Date','Hour'])
rangering = rangering.drop(columns = ['Date','Hour','DayOrNight'])
rangering_modified

,BrukerID,FilmID,Rangering,Tidstempel,DayOrNight
0,0,1561,4,959441640.0,1
1,0,1540,3,959441640.0,1
2,0,88,3,959441640.0,1
3,0,620,4,959441640.0,1
4,0,3771,5,959442113.0,1
...,...,...,...,...,...
900182,6040,1153,4,976584194.0,1
900183,6040,3714,4,976584260.0,1
900184,6040,2834,5,976584260.0,1
900185,6040,48,5,976584300.0,1


In [17]:
#Moving on to the movie dataset, here I want to create a column for each genre, with binary values.
#I also need to check the data in MovieID's to ensure there are no problems.
genre_dummies = pd.get_dummies(film.Sjanger.str.split('|',expand = True).stack(dropna = False)).sum(level = 0)
genre_dummies
film = pd.concat([film.drop(columns = 'Sjanger'), genre_dummies], axis = 1)
film

,FilmID,Tittel,Action,Adventure,Animation,Children,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,Ukjennt,War,Western
0,0,Autumn in New York (2000),0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,"Vie est belle, La (Life is Rosey) (1987)",0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Defying Gravity (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Ruthless People (1986),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Portraits Chinois (1996),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Cat People (1982),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3879,3949,"Saltmen of Tibet, The (1997)",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Bride of Re-Animator (1990),0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3881,3951,True Lies (1994),1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
#I notice that a column named 'Ukjennt' has appeared
#Apparently there are some movies with 'unknown' genres
ukjent = film['Ukjennt'].value_counts()
ukjent_id = film[film['Ukjennt'] == 1]['FilmID']
#Checking if the movies with unknown genres has been rated by users
ukjent_rangering = rangering[rangering.FilmID.isin(ukjent_id)]
print(ukjent_id.values)
print(ukjent_rangering.FilmID.value_counts())
#The three movies iwth unknown genres has been rated. I'm thinking of manually adding their genres in order to remove the 'Ukjennt' column
film[film['Ukjennt'] == 1]['Tittel']
#I search for the movies using IMDB and find their respective genres.

[1881 2554 3090]
2554    303
3090     24
1881      2
Name: FilmID, dtype: int64


1852              Railroaded! (1947)
2511    Name of the Rose, The (1986)
3037               Love Jones (1997)
Name: Tittel, dtype: object

In [19]:
#Setting FilmID as index to match structure of sample_data
film.set_index('FilmID',inplace = True)

In [20]:
print(film[film.index == 1881].to_string())
#Adding genres for Railroaded! (1947)
film.loc[1881,'Crime'] = 1
film.loc[1881,'Drama'] = 1
film.loc[1881,'Film-Noir'] = 1

print(film[film.index == 2554].to_string())
#Adding genres for Name of the Rose, The (1986)
film.loc[2554,'Crime'] = 1
film.loc[2554,'Drama'] = 1
film.loc[2554,'Mystery'] = 1

print(film[film.index == 3090].to_string())
#Adding genres for Love Jones (1997)
film.loc[3090,'Drama'] = 1
film.loc[3090,'Romance'] = 1

film[film['Ukjennt'] == 1]

                    Tittel  Action  Adventure  Animation  Children  Children's  Comedy  Crime  Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  Thriller  Ukjennt  War  Western
FilmID                                                                                                                                                                                                           
1881    Railroaded! (1947)       0          0          0         0           0       0      0            0      0        0          0       0        0        0        0       0         0        1    0        0
                              Tittel  Action  Adventure  Animation  Children  Children's  Comedy  Crime  Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  Thriller  Ukjennt  War  Western
FilmID                                                                                                                                                

,Tittel,Action,Adventure,Animation,Children,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,Ukjennt,War,Western
FilmID,,,,,,,,,,,,,,,,,,,,,
1881,Railroaded! (1947),0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
2554,"Name of the Rose, The (1986)",0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,0,0,1,0,0
3090,Love Jones (1997),0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0


In [21]:
film.drop(columns = 'Ukjennt', inplace = True)
film

,Tittel,Action,Adventure,Animation,Children,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
FilmID,,,,,,,,,,,,,,,,,,,,
0,Autumn in New York (2000),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,"Vie est belle, La (Life is Rosey) (1987)",0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Defying Gravity (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Ruthless People (1986),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Portraits Chinois (1996),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Cat People (1982),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3949,"Saltmen of Tibet, The (1997)",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3950,Bride of Re-Animator (1990),0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [22]:
print(pd.isna(film['Tittel']).value_counts())
# There are no missing values in the Tittel column
# Checking for duplicate values
tittel_unq = film['Tittel'].value_counts()
filmid_unq = film.index.value_counts()
print((filmid_unq > 1).where(lambda x : x == True).dropna())
print((tittel_unq > 1).where(lambda x : x == True).dropna())
# There are no duplicate values, the movie dataset has the same structure as in sample_data and is ready for further analysis

False    3883
Name: Tittel, dtype: int64
Series([], Name: FilmID, dtype: float64)
Series([], Name: Tittel, dtype: float64)


In [23]:
#Fixing the Children genre. At the moment we have both Children and Children's
filmcop = film.copy()
filmcop['Chill'] = film.apply(lambda row: 1 if (row.Children == 1 or row["Children's"] == 1) else 0, axis = 1)
film.drop(columns = ['Children', "Children's"], inplace = True)
film["Children's"] = filmcop.Chill

In [24]:
#we know from earlier analysis that the genres Children's and Animation are highly correlated. Adding this as a new feature
film['Animation | Children'] = film.apply(lambda row: 1 if (row.Animation == 1 and row["Children's"] == 1) else 0, axis = 1)

# Testing new features on my Content Based Filtering Model

In [25]:
userrank_df = rangering_modified.merge(bruker,left_on = 'BrukerID', right_on = 'BrukerID')
final_df = userrank_df.merge(film,left_on = 'FilmID', right_on = 'FilmID')

In [26]:
svr_new_val = []
svr_new_gen = []
for uid in bruker.index.unique():
    X = final_df.loc[final_df['BrukerID'] == uid].drop(columns = ['BrukerID','Tittel','Tidstempel','Postkode','Rangering','Postkode','FilmID','Kjonn','Jobb','Alder']).values
    y = final_df.loc[final_df['BrukerID'] == uid].Rangering.values
    
    X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest,test_size=0.5, random_state=42)
    svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.6))
    svr.fit(X_train,y_train)
    svr_new_val.append(np.sqrt(mean_squared_error(y_val, np.clip(svr.predict(X_val),1,5))))
    svr_new_gen.append(np.sqrt(mean_squared_error(y_test, np.clip(svr.predict(X_test),1,5))))

In [27]:
print('The validation rmse with new features is:' ,pd.DataFrame(svr_new_val).mean()[0])
print('\nThe estimated generalization error of the model with new features is: ',pd.DataFrame(svr_new_gen).mean()[0])

The validation rmse with new features is: 0.9837799398364625

The estimated generalization error of the model with new features is:  0.9887532260572068


In [28]:
svr_val = []
svr_gen = []
for uid in bruker.index.unique():
    X = final_df.loc[final_df['BrukerID'] == uid].drop(columns = ['BrukerID','Tittel','Tidstempel','Postkode','Rangering','Postkode','FilmID','DayOrNight','Animation | Children','Kjonn','Jobb','Alder']).values
    y = final_df.loc[final_df['BrukerID'] == uid].Rangering.values
    
    X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest,test_size=0.5, random_state=42)
    svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.6))
    svr.fit(X_train,y_train)
    svr_val.append(np.sqrt(mean_squared_error(y_val, np.clip(svr.predict(X_val),1,5))))
    svr_gen.append(np.sqrt(mean_squared_error(y_test, np.clip(svr.predict(X_test),1,5))))

In [29]:
print('The mean RMSE for validation data is:' ,pd.DataFrame(svr_val).mean()[0])
print('\nThe estimated generalization error of the model is: ',pd.DataFrame(svr_gen).mean()[0])

The mean RMSE for validation data is: 0.987561178847728

The estimated generalization error of the model is:  0.9912775220257729


**The new features did not yield any significant improvent over the standard model**

In [30]:
#export cleaned dataframes to csv
bruker.to_csv('bruker_cleaned.csv',index = True)
rangering_modified.to_csv('rangering_modified_cleaned.csv', index = False)
rangering.to_csv('rangering_cleaned.csv',index = False)
film.to_csv('film_cleaned.csv', index = True)